In [24]:
#link data :https://www.kaggle.com/datasets/wanderdust/skin-lesion-analysis-toward-melanoma-detection
#import Libraries
import numpy as np 
import matplotlib as plt
import tensorflow as tf 
from tensorflow.keras import layers ,models ,Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,load_img ,img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input , decode_predictions,ResNet50
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D

In [3]:
#Putting data into variables
img_height, img_width = (64,64)
batch_size= 32
train_data = f"data/train"
test_data = f"data/test"

In [4]:
#Divide into testing, training and validation
train_data_1 = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  validation_split=0.4)
train_img = train_data_1.flow_from_directory(train_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical")
test_img = train_data_1.flow_from_directory(test_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical")

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [5]:
x,y = test_img.next()
x.shape

(32, 64, 64, 3)

In [17]:
#Creating and training neural networks
model_cs = ResNet50(include_top=False,weights="imagenet")
x = model_cs.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation="relu")(x)
pred = Dense(train_img.num_classes,activation='softmax')(x)
model = Model(inputs=model_cs.input,outputs=pred)
for layer in model_cs.layers:
    layer.trainable = False
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_img, epochs=10)

Epoch 1/10
83/83 [==============================] - 23s 229ms/step - loss: 1.1989 - accuracy: 0.7892
Epoch 2/10
83/83 [==============================] - 18s 219ms/step - loss: 0.3112 - accuracy: 0.8559
Epoch 3/10
83/83 [==============================] - 18s 219ms/step - loss: 0.2933 - accuracy: 0.8673
Epoch 4/10
83/83 [==============================] - 18s 217ms/step - loss: 0.2762 - accuracy: 0.8703
Epoch 5/10
83/83 [==============================] - 18s 216ms/step - loss: 0.2570 - accuracy: 0.8866
Epoch 6/10
83/83 [==============================] - 18s 214ms/step - loss: 0.2394 - accuracy: 0.8889
Epoch 7/10
83/83 [==============================] - 18s 216ms/step - loss: 0.2292 - accuracy: 0.8950
Epoch 8/10
83/83 [==============================] - 18s 216ms/step - loss: 0.2141 - accuracy: 0.9109
Epoch 9/10
83/83 [==============================] - 18s 217ms/step - loss: 0.1960 - accuracy: 0.9135
Epoch 10/10
83/83 [==============================] - 18s 216ms/step - loss: 0.1941 - accura

In [18]:
model.evaluate(test_img)

21/21 [==============================] - 5s 202ms/step - loss: 0.3560 - accuracy: 0.8439


[0.3559737205505371, 0.8439394235610962]

In [19]:
cnn = Sequential([
                  layers.Conv2D(32, (2,2), activation='relu', input_shape=(64, 64, 3)),
                  layers.MaxPooling2D(3,3),
                  layers.Conv2D(64, kernel_size=(3,3),activation='relu',kernel_initializer='he_normal'),
                  layers.MaxPooling2D(2,2),
                  #layers.Conv2D(64, (2, 2), activation='relu'),
                  layers.Conv2D(64, kernel_size=(2,2),activation='relu',kernel_initializer='he_normal'),
                  layers.MaxPooling2D(2,2),
                  layers.Flatten(),
                  layers.Dense(128,activation='relu'),
                  layers.Dense(2,activation='softmax',kernel_initializer='glorot_normal')
                  
])
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
cnn.fit(train_img, epochs=6)

Epoch 1/6
83/83 [==============================] - 9s 104ms/step - loss: 2.6170 - accuracy: 0.7080
Epoch 2/6
83/83 [==============================] - 9s 104ms/step - loss: 0.4426 - accuracy: 0.7842
Epoch 3/6
83/83 [==============================] - 9s 105ms/step - loss: 0.3986 - accuracy: 0.8195
Epoch 4/6
83/83 [==============================] - 9s 104ms/step - loss: 0.3918 - accuracy: 0.8172
Epoch 5/6
83/83 [==============================] - 9s 104ms/step - loss: 0.3640 - accuracy: 0.8301
Epoch 6/6
83/83 [==============================] - 9s 109ms/step - loss: 0.3498 - accuracy: 0.8366


In [20]:
cnn.evaluate(test_img)

21/21 [==============================] - 2s 88ms/step - loss: 0.3812 - accuracy: 0.8061


[0.3811641335487366, 0.8060606122016907]

In [46]:
test_image = load_img('54.jpg', target_size = (64, 64))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
#training_set.class_indices
if result[0][0] == 1:
    prediction = 'benign'
else:
    prediction = 'malignant'
prediction

1/1 [==============================] - 0s 16ms/step


'malignant'

In [42]:
test_image = load_img('3.jpg', target_size = (64, 64))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
#training_set.class_indices
if result[0][0] == 1:
    prediction = 'benign'
else:
    prediction = 'malignant'
prediction

1/1 [==============================] - 0s 15ms/step


'malignant'